In [0]:
import cv2

In [0]:
# Detect if we are in Google Colaboratory
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
# Determine the locations of auxiliary libraries and datasets.
# `AUX_DATA_ROOT` is where 'notmnist.py', 'animation.py' and 'tiny-imagenet-2020.zip' are.
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgotten to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.insert(0, str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
![ ! -d "focus_level_dataset" ] && unzip -q "{AUX_DATA_ROOT / 'focus_level_dataset.zip'}" -d focus_level_dataset

In [0]:
import os
from PIL import Image
import numpy as np

from torch.utils.data import Dataset, DataLoader

import albumentations
from albumentations.pytorch.transforms import ToTensor
from albumentations import (
    Compose,
    RandomRotate90,
    VerticalFlip,
    HorizontalFlip,
    RandomCrop,
    CenterCrop,
    Normalize
)

In [0]:
CROP_SIZE = 84

class FocusLevelDatasetValidation(Dataset):
    def __init__(self, focused_path, unfocused_path, transform):
        self.images = []
        self.labels = []
        self.transform = transform

        for img in sorted(os.listdir(focused_path)):
          img_path = os.path.join(focused_path, img)
          img = Image.open(img_path).convert('L')
          self.images.append(img)
          self.labels.append(0)
        
        for img in sorted(os.listdir(unfocused_path)):
          img_path = os.path.join(unfocused_path, img)
          img = Image.open(img_path).convert('L')
          self.images.append(img)
          self.labels.append(1)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
      img = np.array(self.images[idx])
      img = self.transform(image=img)['image'].T.unsqueeze(0)
      return img, self.labels[idx]


test_transform = Compose([
    Normalize([0.5], [0.5]),
    ToTensor(),
])

val_dataset = FocusLevelDatasetValidation('./focus_level_dataset/focus_level_dataset/validation/focused', './focus_level_dataset/focus_level_dataset/validation/unfocused', transform=test_transform)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [0]:
class FocusLevelDataset(Dataset):
    def __init__(self, data_path, transform, crop_size=84):
        self.images = []
        self.labels = []
        self.masks = []
        self.width = 768
        self.height = 1024
        self.crop_size = crop_size
        self.transform = transform
        self.random_crop = CenterCrop(self.crop_size, self.crop_size, p=1.0)

        for dir_name in os.listdir(data_path):
            for dir_path, dir_names, file_names in os.walk(os.path.join(data_path, dir_name)):
                if len(dir_names) == 0:       # We are inside folder with specimen

                    # Add grayscaled images
                    for frame in (sorted(file_names, reverse=True)[:10]):
                        img = Image.open(os.path.join(dir_path, frame)).convert('L')
                        self.images.append(img)

                    # Add labels -- 0 means the most focused image
                    self.labels += list(range(10))

                    # Extracting mask for every scpecimen
                    most_blurred_image = cv2.imread(os.path.join(dir_path, sorted(file_names)[0]), 0)
                    _, mask = cv2.threshold(np.array(most_blurred_image), 20, 255, cv2.THRESH_BINARY_INV)
                    for _ in range(len(file_names)):
                        self.masks.append(mask == 0)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = np.array(self.images[idx])
        mask = np.array(self.masks[idx])

        cropped = self.random_crop(image=img, mask=mask)
        img, mask = cropped['image'], cropped['mask']
        img = self.transform(image=img)['image'].T.unsqueeze(0)

        label = self.labels[idx]
        mean_mask = mask.mean()
        if mean_mask < 0.5: # If more than 50% of image is not in mask, image will label as not focused
            label = 9
        elif mean_mask < 0.75: # If more than 75% of image is not in mask, image will label as focused twice worse
            label = min(9, label + label / 2)

        return img, int(label), mask

In [0]:
CROP_SIZE = 84
BATCH_SIZE = 1

transform = Compose([
    Normalize([0.5], [0.5]),
    ToTensor(),
])

dataset = FocusLevelDataset("/content/focus_level_dataset/focus_level_dataset", transform, CROP_SIZE)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

In [0]:
def MLOG(img):
    return np.max(cv2.convertScaleAbs(cv2.Laplacian(img, 3)))

def VOLL4(gray_img):
    tmp_image = np.zeros(gray_img.shape, dtype=int)
    tmp_image[:, :tmp_image.shape[1] - 1] = gray_img[:, 1:]
    sum1 = np.multiply(gray_img, tmp_image).sum()

    tmp_image = np.zeros(gray_img.shape, dtype=int)
    tmp_image[:, :tmp_image.shape[1] - 2] = gray_img[:, 2:]
    sum2 = np.multiply(gray_img, tmp_image).sum()

    return sum1 - sum2

def LAPM(img):
    kernel = np.array([-1, 2, -1])
    laplacianX = np.abs(cv2.filter2D(img, -1, kernel))
    laplacianY = np.abs(cv2.filter2D(img, -1, kernel.T))
    return np.mean(laplacianX + laplacianY)


def TENG(img):
    gaussianX = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    gaussianY = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    return np.mean(gaussianX * gaussianX + gaussianY * gaussianY)


In [0]:
max_acc = 0
max_ind = 0

in_focus = []
for x, y, _ in dataloader:
  if y == 0:
    in_focus.append(VOLL4(x[0].numpy()))

print(min(in_focus), (in_focus))
thres_bin = np.linspace(0.0, max(in_focus), 100)

for ind in range(100):
  acc = 0
  for x, y, _ in dataloader:
    focus_measure = VOLL4(x[0].numpy())
    pred_label = int(thres_bin[ind] > focus_measure)
    
    if pred_label == (int(y) > 5):
      acc += 1
  if acc > max_acc:
    max_ind = ind
    max_acc = acc
  print(thres_bin[ind], acc / len(dataloader))

print(max_ind, max_acc)

0.0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6
0.0 0.6


KeyboardInterrupt: ignored

In [0]:
thresh = thres_bin[max_ind]

acc = 0
for x, y in val_dataloader:
  focus_measure = TENG(x[0].numpy())
  pred_label = int(thresh > focus_measure)
  
  if pred_label == (int(y) > 5):
    acc += 1

print('Accuracy on validation dataset: ', acc / len(val_dataloader))

Accuracy on validation dataset:  0.5584415584415584
